In [1]:
# Install Open CV
# Import cv2 and verify
# python3 -m pip install opencv-python
import cv2 as cv
import numpy as np
from PIL import Image
from stockfish import Stockfish
import config

print(cv.__version__)

4.8.0


In [2]:
def display_matches(img, rects, x_incr, y_incr, pieceName):
    """
    Display all potential matches of template
    """
    piece_notation = []

    for r in rects:
        cv.rectangle(img, (r[0], r[1]), (r[0] + r[2], r[1] + r[3]), (0, 0, 255), 2)
        x = int((r[0] + r[0] + r[2]) / 2)
        y = int((r[1] + r[1] + r[3]) / 2)
        cv.circle(img, (x,y), 10, (0, 0, 255), 2)

        

        notation = pieceName + ""
        if x >= 0 and x <= x_incr:
            notation = notation + "a"
        elif x > x_incr and x <= 2*x_incr:
            notation = notation + "b"
        elif x > 2*x_incr and x <= 3*x_incr:
            notation = notation + "c"
        elif x > 3*x_incr and x <= 4*x_incr:
            notation = notation + "d"
        elif x > 4*x_incr and x <= 5*x_incr:
            notation = notation + "e"
        elif x > 5*x_incr and x <= 6*x_incr:
            notation = notation + "f"
        elif x > 6*x_incr and x <= 7*x_incr:
            notation = notation + "g"
        elif x > 7*x_incr and x <= 8*x_incr:
            notation = notation + "h"
        
        if y >= 0 and y <= y_incr:
            notation = notation + "8"
        elif y > y_incr and y <= 2*y_incr:
            notation = notation + "7"
        elif y > 2*y_incr and y <= 3*y_incr:
            notation = notation + "6"
        elif y > 3*y_incr and y <= 4*y_incr:
            notation = notation + "5"
        elif y > 4*y_incr and y <= 5*y_incr:
            notation = notation + "4"
        elif y > 5*y_incr and y <= 6*y_incr:
            notation = notation + "3"
        elif y > 6*y_incr and y <= 7*y_incr:
            notation = notation + "2"
        elif y > 7*y_incr and y <= 8*y_incr:
            notation = notation + "1"
        

        #print("XY Notation", x, y, notation)
        piece_notation.append(notation)
    return piece_notation
 



def check_color(img, temp, rect):
    """
    Determine if the piece is the same color as the given template
    Need to be very strict on bounds
    """
    y0, y1 = rect[1], rect[1] + rect[3]
    x0, x1 =  rect[0], rect[0] + rect[2]    
    template = (img[y0 : y1, x0 : x1]).copy()
    diff = cv.absdiff(temp, template)
    img_avg = cv.mean(diff)[0] / 255        
    return img_avg < 0.25

def find_template_multiple(img, temp):
    """
    Locate all matching templates
    """
    rects = []
    w, h = temp.shape[1], temp.shape[0]

    result = cv.matchTemplate(img, temp, cv.TM_CCOEFF_NORMED)
    threshold = 0.7 # matching threshold, relatively stable.
    loc = np.where( result >= threshold)

    for pt in zip(*loc[::-1]):
        rects.append((pt[0], pt[1], w, h))    

    # Non-max suppression
    rects, _ = cv.groupRectangles(rects, 1, 1)
    rects = [r for r in rects]   

    return rects

In [4]:
def pieceDetection(img_board, img_piece, pieceName):
    # Pre-processing
    board_x, board_y, color = img_board.shape
    x_incrementer = board_x / 8
    y_incrementer = board_y / 8

    # Convert images to grayscale
    img_board_gray = cv.cvtColor(img_board, cv.COLOR_BGR2GRAY)
    img_piece_gray = cv.cvtColor(img_piece, cv.COLOR_BGR2GRAY)

    # Morpholigcal erosion for stabilization
    s = 2
    kernel = np.ones((s, s), np.uint8)
    img_board_gray_erode = cv.erode(img_board_gray,kernel,iterations = 1)
    img_piece_gray_erode = cv.erode(img_piece_gray,kernel,iterations = 1)


    # All instances of matching templates
    rects = find_template_multiple(img_board_gray_erode, img_piece_gray_erode)

    # Matching color as given input
    matching_color_list = [check_color(img_board_gray, img_piece_gray, r) for r in rects]

    # Keep only matching color rectangles
    matching_color_rects = [r for (r, is_matching) in zip(rects, matching_color_list) if is_matching]

    piece_notation = display_matches(img_board, matching_color_rects, x_incrementer, y_incrementer, pieceName)
    return piece_notation


def display(img_board):
    cv.imshow('Result', img_board)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [5]:
# FEN notation conversion
def fenConverter(board, notations):
    # List of positions ['bc8', 'bf8']
    horizontal = {
        "a": 0,
        "b": 1,
        "c": 2,
        "d": 3, 
        "e": 4,
        "f": 5,
        "g": 6,
        "h": 7
    }

    for notation in notations:
        piece = notation[0]
        row = horizontal[notation[1]]
        col = 8 - int(notation[2])
        board[col][row] = piece

    return board

def condense_list(input_list):
    result = []
    current_sum = 0

    for item in input_list:
        if isinstance(item, int):
            current_sum += item
        else:
            if current_sum != 0:
                result.append(str(current_sum))
            result.append(str(item))
            current_sum = 0

    # Append the last sum if the list ends with integers
    if current_sum != 0:
        result.append(str(current_sum))

    return ''.join(result)


def condenseBoard(board):
    FENlist = []
    for row in board:
        rowStr = condense_list(row)
        FENlist.append(rowStr)
    
    output = ""
    for row in FENlist:
        output = output + row + '/'
    
    return output[:-1]

In [6]:
# Apply everything to ALL available pieces
boardNotation = []

# Trained on specific dimensions for chess board
image = Image.open("Test1.png")
new_image = image.resize((1552, 1554))
new_image.save("Test1RSZ.png")

img_board = cv.imread("Test1RSZ.png")


# Load in all images being used
b = (cv.imread('BBishop.png'), "b")
k = (cv.imread('BKing.png'), 'k')
n = (cv.imread('BKnight.png'), 'n')
p = (cv.imread('BPawn.png'), 'p')
q = (cv.imread('BQueen.png'), 'q')
r = (cv.imread('BRook.png'), 'r')

B = (cv.imread('WBishop.png'), 'B')
K = (cv.imread('WKing.png'), 'K')
N = (cv.imread('WKnight.png'), 'N')
P = (cv.imread('WPawn.png'), 'P')
Q = (cv.imread('WQueen.png'), 'Q')
R = (cv.imread('WRook.png'), 'R')

pieces = [b,k,n,p,q,r,B,K,N,P,Q,R]


for piece in pieces:
    tempNotation = pieceDetection(img_board, piece[0], piece[1])
    boardNotation.append(tempNotation)


board = [[1]*8 for i in range(8)]
for pieceNotation in boardNotation:
    fenConverter(board, pieceNotation)

FEN = condenseBoard(board)
print(FEN)
display(img_board)

# Parameters of FEN(Piece Placement, Active Color, Castling Rights, Possible En Passant Targets, Halfmove Clock, Fullmove Number)
# Determine who goes next? w or b
# Castling rights? Qk (white can castle Queen side and black can castle king side)
# En passant target (-) 
# Number of half moves made <100 (invalid)
# Number of full moves; incremeneted by one every time black moves
# EX: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR W KQkq - 0 0

# rnbqkbnr/pppp1ppp/4p3/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 0 2
# rnbqkbnr/pppp1ppp/4p3/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 0 2

#display(img_board)

rnbqkbnr/pppp1ppp/4p3/8/3P4/8/PPP1PPPP/RNBQKBNR


In [8]:
FENstr = "rnbqkbnr/pppp1ppp/4p3/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 0 2"
stockfishPath = config.stockfishPath #"C:\\Users\kaleb\Downloads\stockfish\stockfish-windows-x86-64-avx2"


def bestMove(FENstr, stockfishPath):
    stockfish = Stockfish(stockfishPath)
    if stockfish.is_fen_valid(FENstr):
        stockfish.set_fen_position(FENstr)
        return stockfish.get_best_move_time(1000) # in ms
    else:
        return "Invalid FEN notation"

bestMove(FENstr, stockfishPath)

'e2e4'